## Pengenalan

##### Final project kali ini saya ingin mengambil topik Image Classification, yaitu ingin memprediksi antara gambar kendaraan atau bukan kendaraan.

##### Dataset yang digunakan diambil dari kaggle, ada sekitar 17.ooo gambar dengan format png (64x64x3).

##### Ada 2 label/kelas, yaitu non-vehicles dan vehicles.

## Import Pustaka

##### Deep learning framework yang digunakan adalah TensorFlow, jadi pustaka yang saya import adalah TensorFlow.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Data Loading

##### Pertama, saya download datanya terlebih dahulu di kaggle: https://www.kaggle.com/brsdincer/vehicle-detection-image-set

##### Kedua, saya masukan data ke dalam google drie saya: https://drive.google.com/file/d/12tOThhpWgxuaR4Ism9Y_XbYsl7KeqdiC/view?usp=sharing

##### Lalu saya mount google drive dan load dataset tersebut. Data harus di download terlebih dahulu (gunakan salah satu link di atas).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/data.zip

##### Di dalam folder data, terdapat 2 subfolder yaitu non-vehicles dan vehicles

In [ ]:
!ls data

## Generate Dataset

##### Data Preprocessing, pembagian data menjadi train dan val.

In [ ]:
image_size = (180, 180)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

## Visualisasi Dataset

##### Dengan menggunakan pustaka matplotlib untuk memvisualisasikan dataset.
##### Label 0 untuk "non-vehicles"
##### Label 1 untuk "vehicles"

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

## Data Preprocessing

##### Penggunaan data augmentation untuk dataset yang tidak terlalu besar.

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

##### Konfigurasi dataset untuk performance.

In [ ]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

## Pendefinisian model

##### Model dimulai dengan data preprocessiong, yaitu data augmentation diikuti dengan rescaling layer.

In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

## Pelatihan Model

In [ ]:
epochs = 15

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
r = model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

## Evaluasi Model

In [ ]:
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])

In [ ]:
plt.plot(r.history['accuracy'])
plt.plot(r.history['val_accuracy'])

## Model Inference

In [ ]:
img = keras.preprocessing.image.load_img(
    "data/vehicles/5000.png", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = predictions[0]
print(
    "This image is %.2f percent non-vehicles and %.2f percent vehicles."
    % (100 * (1 - score), 100 * score)
)

## Kesimpulan

##### Di sini kita menemukan bahwa model bekerja cukup baik dengan akurasi sekitar 99%. Jadi model ini tepat untuk mengidentifikasi kendaraan.